Write an elementary Python program that creates a present value, future value “calculator.” The program
should query the user for the following:
● Present value or future value calculation
● Number of “discrete forecast” periods
● Number of distinct “growth” phases (including “none”), and for each phase:
o Applicable growth rate
o Length of phase
o And, if it is the last growth phase, whether it is an annuity or perpetuity (only the last phase
can be either an annuity or perpetuity; all others must be annuities)
● Applicable discount rate
● And then return the appropriate present value or future value
● Recommendation: It might be a good idea to show how it was constructed, for example:
o PV Discrete Forecast: $100
o PV Phase I Annuity: $80
o PV Phase II Annuity: $70
o PV Phase III Perpetuity: $500
o TOTAL PV = $750
● HINT: If you are using deferred annuity or perpetuity formulas, you need to take the PV of the PV
of the deferred annuity or perpetuity

In [56]:
def get_inputs():
    print("This is a PV/FV Calculator")
    print("Type 'exit' at any prompt to quit the program.")
    

    # If the user types exit the program will return None and stop.
    # If the user types an invalid input the program will ask them to type it in again.
    while True:
        calc_type = input("Calculate PV or FV? (PV/FV): ").upper()
        if calc_type == 'EXIT':
            return None
        if calc_type in ['PV', 'FV']:
            break
        print("Invalid option. Please enter 'PV' or 'FV'.")

    while True:
        user_input = input("Enter discount rate (decimal): ")
        if user_input.upper() == 'EXIT':
            return None
        try:
            r = float(user_input)
            break
        except ValueError:
            print("Invalid input. Please enter a valid number for the discount rate.")

    while True:
        user_input = input("Number of discrete forecast periods: ")
        if user_input.upper() == 'EXIT':
            return None
        try:
            num_discrete_periods = int(user_input)
            if num_discrete_periods < 0:
                print("Number of periods must be a whole number. Please try again.")
                continue
            break
        except ValueError:
            print("Invalid input. Please enter a valid positive integer for the number of periods.")

    cash_flow_list = []
    for i in range(1, num_discrete_periods + 1):
        while True:
            user_input = input(f"Cash flow in period {i}: ")
            if user_input.upper() == 'EXIT':
                return None
            try:
                cf = float(user_input)
                cash_flow_list.append(cf)
                break
            except ValueError:
                print("Invalid input. Please enter a valid number for the cash flow.")

    while True:
        user_input = input("Number of growth phases: ")
        if user_input.upper() == 'EXIT':
            return None
        try:
            num_growth_phases = int(user_input)
            if num_growth_phases < 0:
                print("Number of growth phases cannot be negative. Please try again.")
                continue
            break
        except ValueError:
            print("Invalid input. Please enter a valid non-negative integer for the number of growth phases.")

    phases = []
    is_perpetuity = False
    for phase_num in range(1, num_growth_phases + 1):
        while True:
            user_input = input(f"Growth rate for phase {phase_num} (decimal): ")
            if user_input.upper() == 'EXIT':
                return None
            try:
                g = float(user_input)
                break
            except ValueError:
                print("Invalid input. Please enter a valid number for the growth rate.")
        
        while True:
            user_input = input(f"Length of phase {phase_num} (number of periods): ")
            if user_input.upper() == 'EXIT':
                return None
            try:
                phase_length = int(user_input)
                if phase_length <= 0:
                    print("Phase length must be positive. Please try again.")
                    continue
                break
            except ValueError:
                print("Invalid input. Please enter a valid positive integer for the phase length.")
        
        if phase_num == num_growth_phases:
            while True:
                user_input = input("Annuity or Perpetuity for last phase? (A/P): ").upper()
                if user_input == 'EXIT':
                    return None
                if user_input in ['A', 'P']:
                    if user_input == 'P':
                        is_perpetuity = True
                        ap_type = 'P'
                    else:
                        ap_type = 'A'
                    break
                print("Invalid option. Please enter 'A' or 'P'.")
        else:
            ap_type = 'A'
        phases.append({'g': g, 'length': phase_length, 'type': ap_type})

    if num_growth_phases > 0:
        while True:
            user_input = input(f"Enter the cash flow for period {num_discrete_periods + 1}: ")
            if user_input.upper() == 'EXIT':
                return None
            try:
                first_phase_cf = float(user_input)
                break
            except ValueError:
                print("Invalid input. Please enter a valid number for the cash flow.")
    else:
        first_phase_cf = 0.0

    if calc_type == 'FV' and is_perpetuity:
        print("You can't get a FV with a perpetuity. Please restart and choose PV or an annuity.")
        return get_inputs()

    total_periods = num_discrete_periods
    for phase in phases:
        total_periods += phase['length']

    return {
        'calc_type': calc_type,
        'r': r,
        'num_discrete_periods': num_discrete_periods,
        'cash_flow_list': cash_flow_list,
        'n_phases': num_growth_phases,
        'phases': phases,
        'first_phase_cf': first_phase_cf,
        'total_periods': total_periods
    }

Here I make a function to seperate my discrete cash flows

In [57]:
# Function to Calculate Discrete Part
def calculate_discrete(inputs):
    # Give me all the values
    calc_type = inputs['calc_type']
    r = inputs['r']
    num_discrete_periods = inputs['num_discrete_periods']
    cash_flow_list = inputs['cash_flow_list']
    total_periods = inputs['total_periods']
    
    # If its a PV
    if calc_type == 'PV':
        pv_discrete = 0.0 
        # Here I go through each discrete period and discount every cash flow back to PV with the formula
        for t in range(1, num_discrete_periods + 1):
            pv_discrete += cash_flow_list[t - 1] / (1 + r) ** t
        return pv_discrete
    # If its an FV
    else:
        fv_discrete = 0.0
        # Here I loop through every period and compound every cash flow forward
        # to the last period bringing every discrete cash flow to the future end point.
        for t in range(1, num_discrete_periods + 1):
            fv_discrete += cash_flow_list[t - 1] * (1 + r) ** (total_periods - t)
        return fv_discrete

Here I made some helper functions to clean up my code. I check r == g to avoid dividing by 0.

In [58]:
def pv_growing_annuity(pmt, r, g, n):
    if r == g:
        return pmt * n / (1 + r)
    return pmt * (1 - ((1 + g) / (1 + r)) ** n) / (r - g)

In [59]:
def pv_growing_perpetuity(pmt, r, g):
    return pmt / (r - g) if r > g else float('inf')

In [60]:
def fv_growing_annuity(pmt, r, g, n):
    if r == g:
        return pmt * n * (1 + r) ** (n - 1)
    return pmt * ((1 + r) ** n - (1 + g) ** n) / (r - g)

In [61]:
# Function to Calculate Growth Phases
def calculate_growth_phases(inputs, discrete_value):
    calc_type = inputs['calc_type']
    r = inputs['r']
    phases = inputs['phases']
    first_phase_cf = inputs['first_phase_cf']
    num_discrete_periods = inputs['num_discrete_periods']
    total_periods = inputs['total_periods']
    
    components = {'Discrete Forecast': discrete_value}
    total = discrete_value
    
    # Here is the starting cash flow and time
    current_cf = first_phase_cf
    current_time = num_discrete_periods + 1
    
    # Here we go through each phase
    for phase_index, phase in enumerate(phases):
        g = phase['g']
        length = phase['length']
        phase_type = phase['type']
        
        if calc_type == 'PV':
            # This is for a annuity
            if phase_type == 'A':
                # I am calculating the PV of a deferred anuity
                # This calculates the PV of the growing annuity as if it started at the beginning of the phase
                pv_start = pv_growing_annuity(current_cf, r, g, length)
                # This discounts pv_start back to t = 0 through this equation
                pv_phase = pv_start / (1 + r) ** (current_time - 1)
            # This is for a perpetuity
            else:
                pv_start = pv_growing_perpetuity(current_cf, r, g)
                # Same logic as the annuity
                pv_phase = pv_start / (1 + r) ** (current_time - 1)
            components[f'Phase {phase_index + 1} {"Annuity" if phase_type == "A" else "Perpetuity"}'] = pv_phase
            total += pv_phase
        # This is for the FV, no perpetuity for FV
        else:
            fv_end_phase = fv_growing_annuity(current_cf, r, g, length)
            # Get period of phase's last cash flow
            end_phase_time = current_time + length - 1
            periods_to_end = total_periods - end_phase_time
            # This compounds the FV towards the end
            fv_phase = fv_end_phase * (1 + r) ** periods_to_end
            # Hard coded annuity here since there is no perpetuity for FV
            components[f'Phase {phase_index + 1} Annuity'] = fv_phase
            total += fv_phase
        
        # Update for next phase
        current_time += length
        # Update the cash flow if there are more phases
        if phase_index + 1 < len(phases):
            next_g = phases[phase_index + 1]['g']
            current_cf = current_cf * (1 + g) ** (length - 1) * (1 + next_g)
    
    return components, total

In [62]:
# Main Execution
inputs = get_inputs()
discrete_value = calculate_discrete(inputs)
components, total = calculate_growth_phases(inputs, discrete_value)
    
prefix = 'PV' if inputs['calc_type'] == 'PV' else 'FV'
for key, value in components.items():
    print(f"{prefix} {key}: ${value:.2f}")
print(f"TOTAL {prefix} = ${total:.2f}")

This is a PV/FV Calculator
Type 'exit' at any prompt to quit the program.
PV Discrete Forecast: $0.00
PV Phase 1 Annuity: $4738.44
PV Phase 2 Perpetuity: $11880.01
TOTAL PV = $16618.44
